In [1]:
import numpy as np
import random

In [2]:
def createEmptySolution(instance):
    sol = {}
    sol['instance'] = instance
    sol['sol'] = set()
    sol['of'] = 0
    return sol

def addToSolution(sol, u, ofVariation = -1):
    if ofVariation < 0:
        for s in sol['sol']:
            sol['of'] += sol['instance']['d'][u][s]
    else:
        sol['of'] += ofVariation
    sol['sol'].add(u)

def removeFromSolution(sol, u, ofVariation = -1):
    sol['sol'].remove(u)
    if ofVariation < 0:
        for s in sol['sol']:
            sol['of'] -= sol['instance']['d'][u][s]
    else:
        sol['of'] -= ofVariation

def evaluate(sol):
    of = 0
    for s1 in sol['sol']:
        for s2 in sol['sol']:
            if s1 < s2:
                of += sol['instance']['d'][s1][s2]
    return of


def isFeasible(sol):
    return len(sol['sol']) == sol['instance']['p']


def contains(sol, u):
    return u in sol['sol']


def distanceToSol(sol, u, without = -1):
    d = 0
    for s in sol['sol']:
        if s != u and s != without:
            d += sol['instance']['d'][s][u]
    return round(d, 2)


def printSolution(sol):
    print("Solution: ", end="")
    for s in sol['sol']:
        print(s, end=" ")
    print()
    print("Objective Value: "+str(round(sol['of'], 2)))

In [3]:
#from structure import solution

def improve(sol):
    improve = True
    while improve:
        improve = tryImprove(sol)


def tryImprove(sol):
    sel, ofVarSel, unsel, ofVarUnsel = selectInterchange(sol)
    if ofVarSel < ofVarUnsel:
        removeFromSolution(sol, sel, ofVarSel)
        addToSolution(sol, unsel, ofVarUnsel)
        return True
    return False


def selectInterchange(sol):
    n = sol['instance']['n']
    sel = -1
    bestSel = 0x3f3f3f
    unsel = -1
    bestUnsel = 0
    for v in sol['sol']:
        d = distanceToSol(sol, v)
        if d < bestSel:
            bestSel = d
            sel = v
    for v in range(n):
        d = distanceToSol(sol, v, without=sel)
        if not contains(sol, v):
            if d > bestUnsel:
                bestUnsel = d
                unsel = v
    return sel, round(bestSel,2), unsel, round(bestUnsel,2)

In [4]:
# from structure import solution
# import random


def construct(inst, alpha):
    sol = createEmptySolution(inst)
    n = inst['n']
    u = random.randint(0, n-1)
    addToSolution(sol, u)
    cl = createCandidateList(sol, u)
    alpha = alpha if alpha >= 0 else random.random()
    while not isFeasible(sol):
        gmin, gmax = evalGMinGMax(cl)
        threshold = gmax - alpha * (gmax - gmin)
        rcl = []
        for i in range(len(cl)):
            if cl[i][0] >= threshold:
                rcl.append(cl[i])
        selIdx = random.randint(0, len(rcl)-1)
        cSel = rcl[selIdx]
        addToSolution(sol, cSel[1], cSel[0])
        cl.remove(cSel)
        updateCandidateList(sol, cl, cSel[1])
    return sol, alpha


def evalGMinGMax(cl):
    gmin = 0x3f3f3f
    gmax = 0
    for c in cl:
        gmin = min(gmin, c[0])
        gmax = max(gmax, c[0])
    return gmin, gmax


def createCandidateList(sol, first):
    n = sol['instance']['n']
    cl = []
    for c in range(n):
        if c != first:
            d = distanceToSol(sol, c)
            cl.append([d, c])
    return cl


def updateCandidateList(sol, cl, added):
    for i in range(len(cl)):
        c = cl[i]
        c[0] += sol['instance']['d'][added][c[1]]



In [5]:
def pathRelinking(initial_solution, guiding_solution):
    current_solution = initial_solution.copy()
    end_solution = guiding_solution.copy()
    pr_solution = guiding_solution.copy()
    pr_solution_before_Local_Search = guiding_solution.copy()
    intermediate_of_sol2 = {}
    initial_solution2 = {}
    
    # find elements present in initial solution and are also present in guiding solution; 
    
    common_elements = current_solution['sol'].intersection(end_solution['sol'])
    
    # find elements not present in initial solution but are present in guiding solution
    # these elements are used to calculate distance by element-wise replacement in initial solution
    
    uncommon_elements = list(end_solution['sol'] - common_elements)
    
    i = 0
    
    # create empty list that will store index of replaceed element in current solution that is identified as best
    # solution at a given iteration
    
    exclude_index_from_solution = []
    
    #iterations here are determined by number of elements to be replaced (length of uncommon list of elements)
    
    # while there are elements to be replaced in uncommon_elements list
    
    while len(uncommon_elements) > 0:
        
        if i == 0:
            initial_solution = current_solution.copy()
        
        intermediate_of_sol = {}                                       # this holds best solution of each iteration
        
        for u in uncommon_elements:                                    # iterate over the replacing elements
            
            d1 = 0                                                     # distance ("of") of replacing element
            d2 = 0                                                     # distance ("of") of replaced element
            
            for pos in range(len(initial_solution['sol'])):
                if pos not in exclude_index_from_solution:             # exclude every replaced position in best solution to be replaced again
                    
                    initial_solution2 = initial_solution.copy()        # the current best solution of this iteration
                    initial_solution2['sol'] = list(initial_solution2['sol'])
                    
                    d2 = distanceToSol(initial_solution2, initial_solution2['sol'][pos]) # distance of element to be replaced
                    initial_solution2['sol'][pos] = u
                    
                    d1 = distanceToSol(initial_solution2, u)           # distance of element replacing
                    
                    d = initial_solution2['of'] - d2 + d1              # calculate distance ("of") after replacement
                    
                    intermediate_of_sol[d] = initial_solution2['sol']     # the best solution retained, as {objective : solution}, after every iteration
        
        # update objective value and solution of best solution of this iteration as placeholder for next iteration
        
        initial_solution['sol'] = intermediate_of_sol[max(intermediate_of_sol)]
        initial_solution['of'] = max(intermediate_of_sol)
        
        # store all best solution from each iteration, as {Best Objective: Corresponding Solution}
        
        intermediate_of_sol2[round(max(intermediate_of_sol),2)] = intermediate_of_sol[max(intermediate_of_sol)]
        
        # index of the element replaced in the best solution of this iteration
        
        skip_index = [i for i,e in enumerate(initial_solution['sol']) if e in list(set(initial_solution['sol']).
                                                                            intersection(set(uncommon_elements)))]
        
        #add index to skipping list
        
        exclude_index_from_solution.extend(skip_index)
        
        # remove the replacing element (in best solution of this iteration) from uncommon list of elements
        
        uncommon_elements = list(set(uncommon_elements) - set(intermediate_of_sol[max(intermediate_of_sol)]))
        
        
        i+=1
    
    # store the best solution of this Initial and Guiding Pair of solution
    
    pr_solution_before_Local_Search['sol'] = intermediate_of_sol2[max(intermediate_of_sol2)]
    pr_solution_before_Local_Search['of'] = round(max(intermediate_of_sol2),2)
    pr_solution['sol'] = set(intermediate_of_sol2[max(intermediate_of_sol2)])
    pr_solution['of'] = round(max(intermediate_of_sol2),2)
        
    # Apply Local Search to the best solution
    
    improve(pr_solution)

    return pr_solution, pr_solution_before_Local_Search, intermediate_of_sol2

In [6]:
# from constructives import cgrasp_eff, cgrasp
# from localsearch import lsbestimp, lsfirstimp, lsfirstimp_sorted
# from structure import instance, solution

def execute(inst, iters, alpha):
    best_grasp = None
    best_pr = {}
    elite = {}
    pr_solutions = {}
    
    print(f'\n+++++++++++++GRASP SOLUTION WITHOUT PATH RE-LINKING++++++++++++++++++++')
    for i in range(iters):
        print("Iter "+str(i+1)+": \n", end="")
        
        # sol = cgrasp_eff.construct(inst, alpha)
        sol, alpha_grasp = construct(inst, alpha)
        print("C -> "+str(round(sol['of'], 2)), end=", ")
        
        improve(sol)
        # lsfirstimp.improve(sol)
        # lsfirstimp_sorted.improve(sol)
        
        print("LS -> "+str(round(sol['of'], 2)), end=", ")
        print("Alpha -> "+str(round(alpha_grasp,2)))
        
                                           
        if best_grasp is None or best_grasp['of'] < sol['of']:
            best_grasp = sol
        
        # Elite Set Creation
        elite[round(sol['of'],2)] = sol['sol']                      # collect solution of improved GRASP in every iteration
    
    # reordering elite elements from best to least
    dict_Keys = list(elite.keys())
    elite = {i: elite[i] for i in sorted(dict_Keys, reverse= True)} # order the elite set from highest to lowest 'of' value
    elite_elements = list(elite.items())                            # convert the elite set from {Objective : Solution} -> [(Objective, Solution)]
    paired_elements = [(a,b) for idx, a in enumerate(elite_elements) 
                       for b in elite_elements[idx+1:]]             # create distinct pairs of solutions (guiding, initial)
    
    # creating path-relinking solution
    print(f'\n+++++++++++++GRASP SOLUTION WITH PATH RE-LINKING++++++++++++++++++++')
    x = 0
    for i in paired_elements:
        x+=1
        guiding_solution = createEmptySolution(inst)
        guiding_solution['sol'] = i[0][1]
        guiding_solution['of'] = i[0][0]
        initial_solution = createEmptySolution(inst)
        initial_solution['sol'] = i[1][1]
        initial_solution['of'] = i[1][0]
        
        pr_sol_ls, pr_sol, intermediate_sols = pathRelinking(initial_solution, guiding_solution)
        
#         print(f'\nFor PAIR {x} -> \nInitial Solution: {initial_solution["sol"]} \nInitial Objective: {initial_solution["of"]} \nGuiding Solution: {guiding_solution["sol"]} \nGuiding Objective: {guiding_solution["of"]}')
#         print('\nIntermediate Solutions:')
#         for j in range(len(intermediate_sols.items())):
#             print(f'\nAfter Step {j+1}:\nSolution: ',list(intermediate_sols.items())[j][1],
#                   '\nObjective: ',list(intermediate_sols.items())[j][0])
#         print(f'\nFor PAIR {x}:\nBest Solution \nWithout LS:', pr_sol['sol'],'\nObjective: ',pr_sol['of'], '\nWith LS', pr_sol_ls['sol'],'\nObjective: ',pr_sol_ls['of'])
#         print(f'\n+++++++++++++++++++++++++++++++++')

        print(f"PAIR {x}: \n", end="")
        print("Initial -> "+str(round(initial_solution["of"],2)), end=", ")
        print("Guiding -> "+str(round(guiding_solution["of"],2)), end=", ")
        print("Before LS -> "+str(round(pr_sol["of"],2)), end=", ")
        print("After LS -> "+str(round(pr_sol_ls["of"],2)))
        pr_solutions[pr_sol_ls['of']] = pr_sol_ls['sol']
    #print(pr_solutions)
    best_pr['of'] = round(max(pr_solutions),2)
    best_pr['sol'] = pr_solutions[max(pr_solutions)]
    return best_grasp, best_pr


In [9]:
def readInstance(path):
    instance = {}
    with open(path, "r") as f:
        # First line in file has two numbers: n p
        n, p = f.readline().split()
        n = int(n)
        p = int(p)
        instance['n'] = n
        instance['p'] = p
        instance['d'] = []
        for i in range(n):
            instance['d'].append([0] * n)
        for i in range(n):
            for j in range(i+1, n):
                u, v, d = f.readline().split()
                u = int(u)
                v = int(v)
                d = round(float(d), 2)
                instance['d'][u][v] = d
                instance['d'][v][u] = d
    return instance

def executeInstance():
    path = "MDG-a_1_100_m10.txt"
    inst = readInstance(path)
    
    sol_grasp, sol_pr = execute(inst, 10, 1)   #solution from only grasp and grasp with path re-linking
    
    print("==================================================================")
    print("\nBEST GRASP SOLUTION WITHOUT PATH RE-LINKING:")
    printSolution(sol_grasp)
    print("==================================================================")
    print("\nBEST GRASP SOLUTION WITH PATH RE-LINKING:")
    printSolution(sol_pr)
    

In [10]:
random.seed(1)
executeInstance()


+++++++++++++GRASP SOLUTION WITHOUT PATH RE-LINKING++++++++++++++++++++
Iter 1: 
C -> 213.85, LS -> 345.59, Alpha -> 1
Iter 2: 
C -> 232.32, LS -> 336.21, Alpha -> 1
Iter 3: 
C -> 266.56, LS -> 345.16, Alpha -> 1
Iter 4: 
C -> 227.37, LS -> 341.1, Alpha -> 1
Iter 5: 
C -> 207.04, LS -> 329.5, Alpha -> 1
Iter 6: 
C -> 229.4, LS -> 344.04, Alpha -> 1
Iter 7: 
C -> 228.0, LS -> 352.23, Alpha -> 1
Iter 8: 
C -> 252.1, LS -> 337.66, Alpha -> 1
Iter 9: 
C -> 233.74, LS -> 343.13, Alpha -> 1
Iter 10: 
C -> 236.11, LS -> 342.82, Alpha -> 1

+++++++++++++GRASP SOLUTION WITH PATH RE-LINKING++++++++++++++++++++
PAIR 1: 
Initial -> 345.59, Guiding -> 352.23, Before LS -> 352.23, After LS -> 352.23
PAIR 2: 
Initial -> 345.16, Guiding -> 352.23, Before LS -> 352.23, After LS -> 352.23
PAIR 3: 
Initial -> 344.04, Guiding -> 352.23, Before LS -> 352.23, After LS -> 352.23
PAIR 4: 
Initial -> 343.13, Guiding -> 352.23, Before LS -> 352.23, After LS -> 352.23
PAIR 5: 
Initial -> 342.82, Guiding -> 352.